# Notebook to visualize computation of Landmarked Region

### Imports

In [ ]:
import sys
from copy import deepcopy

import cv2
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../")
from src.adnet import compute_landmarks

%load_ext autoreload
%autoreload 2

### Setup

In [ ]:
img_p = "../data/ColorFERET-00472_940519_hr_small_cropped.png"
model_p = "../checkpoints/adnet/adnet_ofiq.onnx"
landmarks = compute_landmarks(img_p, model_p)
# print(landmarks.shape)

# Helper function


def draw_points(img: np.ndarray, points: np.ndarray, name: str = "default"):
    for idx in range(len(points)):
        x, y = points[idx]
        cv2.circle(img, (x, y), 3, (255, 0, 0), cv2.FILLED)
        cv2.putText(img, name, (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    return img


# Show computed landmarks
img = cv2.imread(img_p)
width, height, channels = img.shape
img_landmarks = deepcopy(img)
for idx in range(len(landmarks)):
    x, y = landmarks[idx]
    cv2.circle(img_landmarks, (x, y), 3, (255, 0, 0), cv2.FILLED)
    cv2.putText(img_landmarks, str(idx), (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
plt.imshow(cv2.cvtColor(img_landmarks, cv2.COLOR_BGR2RGB))
plt.savefig("../output/01_landmarks.png", bbox_inches="tight")
plt.show()

### Break down GetFaceMask() function into individual parts

In [ ]:
# OPTIONAL: Select eyes_midpoint, chin and contour landmarks

alpha = 0.0
width, height, channels = img.shape

left_eye_corners = np.stack([landmarks[60], landmarks[64]])
right_eye_corners = np.stack([landmarks[68], landmarks[72]])

eye_corners = np.concatenate([left_eye_corners, right_eye_corners])
eyes_midpoint = np.sum(eye_corners, axis=0) / len(eye_corners)
eyes_midpoint = eyes_midpoint.astype("int")
eyes_midpoint = eyes_midpoint[None, :]  # Add extra dimension

chin = landmarks[16]
chin = chin[None, :]  # Add extra dimension
contour_indices = [0, 7, 25, 32]
contour_points = []
for idx in contour_indices:
    contour_points.append(landmarks[idx])
contour = np.array(contour_points)

# Drawing function
img_out = deepcopy(img)
img_out = draw_points(img_out, eyes_midpoint, "eyes_midpoint")
img_out = draw_points(img_out, chin, "chin")
img_out = draw_points(img_out, contour, "contour")
plt.imshow(cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# OPTIONAL: Fit ellipse to selected landmark points
if alpha > 0.0:
    chin_midpoint_vector = eyes_midpoint - chin
    top_of_forehead = eyes_midpoint + alpha * chin_midpoint_vector

    ellipse_points = np.concatenate([contour, chin, top_of_forehead])
    ellipse_points = np.array(ellipse_points, dtype=np.int32)
    fitted_ellipse = cv2.fitEllipse(ellipse_points)
    center = (int(fitted_ellipse[0][0]), int(fitted_ellipse[0][1]))  # Ellipse center (x, y)
    axes = (
        int(fitted_ellipse[1][0] / 2),
        int(fitted_ellipse[1][1] / 2),
    )  # Semi-major and semi-minor axes
    angle = int(fitted_ellipse[2])  # Rotation angle
    poly_points = cv2.ellipse2Poly(center, axes, angle, 0, 360, 10)

    # Discard ellipse points which are not on forehead
    poly_points_list = []
    chin_midpoint_vector = chin_midpoint_vector.squeeze()
    for p in poly_points:
        if np.dot(p - chin, chin_midpoint_vector) > 1.1 * np.dot(
            chin_midpoint_vector, chin_midpoint_vector
        ):
            poly_points_list.append(p)
    poly_points = np.array(poly_points_list, dtype=np.int32)

    # Add poly_points to landmark points
    # landmarks = np.concatenate([landmarks, poly_points])
    landmarks = landmarks.astype("int")

    # Drawing function
    img_out = draw_points(deepcopy(img), poly_points, "elips")
    plt.imshow(cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# Compute convex hull from selection of landmarks

# Fit convex hull
hull_points = cv2.convexHull(landmarks).squeeze()

# Drawing function
img_hullpoints = draw_points(deepcopy(img), hull_points, "hl")
plt.imshow(cv2.cvtColor(img_hullpoints, cv2.COLOR_BGR2RGB))
plt.savefig("../output/02_convex_hull.png", bbox_inches="tight")
plt.show()

In [ ]:
# Convert hull points to 224x224 mask

rect = cv2.boundingRect(hull_points)
rect_x, rect_y, rect_width, rect_height = rect

b = int(rect_y - rect_height * 0.05)
d = int(rect_y + rect_height * 1.05)
a = int(rect_x + rect_width / 2.0 - (d - b) / 2.0)
c = int(rect_x + rect_width / 2.0 + (d - b) / 2.0)

# Compute relative landmarks on cropped image
img_size = 224
hull_point_list = []
for idx in range(len(hull_points)):
    point = hull_points[idx]
    point = (point - np.array([a, b])) / (d - b) * img_size
    hull_point_list.append(point)
hull_points = np.array(hull_point_list, dtype=np.int32)

# Generate mask from convex hull
mask = np.zeros((img_size, img_size), dtype=np.uint8)
cv2.fillConvexPoly(mask, np.array(hull_points, dtype=np.int32), 1)

# Show mask
plt.imshow(mask, cmap="gray")
plt.show()

In [ ]:
# Resize mask to original size of input image

face_region = np.zeros((height, width), dtype=np.uint8)
mask_rescaled = cv2.resize(mask, (c - a, d - b), interpolation=cv2.INTER_NEAREST)

left, top, right, bottom = 0, 0, mask_rescaled.shape[0], mask_rescaled.shape[1]
an, bn, cn, dn = a, b, c, d

if a < 0:
    left -= a
    an = 0
if c > width:
    right -= c - width
    cn = width
if b < 0:
    top -= b
    bn = 0
if d > height:
    bottom -= d - height
    dn = height

crop = mask_rescaled[top:bottom, left:right]
face_region[bn:dn, an:cn] = crop

# Show mask
plt.imshow(face_region, cmap="grey")
plt.show()

### Addition: Blend mask on input image

In [ ]:
# Plot face_mask on img
face_mask = face_region
face_mask = cv2.cvtColor(face_mask, cv2.COLOR_GRAY2BGR)  # Extend face_mask channels
face_mask[:, :, 2] = face_mask[:, :, 2] * 120.0
img_rgb = cv2.cvtColor(deepcopy(img), cv2.COLOR_BGR2RGB)
img_mask = cv2.addWeighted(img_rgb, 0.7, face_mask, 0.3, 0.0)

plt.imshow(img_mask)
plt.savefig("../output/03_mask_on_img.png", bbox_inches="tight")
plt.show()

In [ ]:
# Create subplot with all steps
from typing import List

fig, axs = plt.subplots(1, 3, figsize=(12, 6))
axs: List[plt.Axes] = np.ravel(axs)

for ax in axs:
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

axs[0].imshow(cv2.cvtColor(img_landmarks, cv2.COLOR_BGR2RGB))
axs[0].set_title("ADNet Landmarks")
axs[1].imshow(cv2.cvtColor(img_hullpoints, cv2.COLOR_BGR2RGB))
axs[1].set_title("OpenCV convexHull")
axs[2].imshow(img_mask)
axs[2].set_title("Landmarked Region")

plt.tight_layout()
plt.savefig("../output/ofiq_convexHull.png", bbox_inches="tight")
plt.show()